## Inputs
---

In [1]:
# reading data and working with arrays
import h5py, nrrd
import numpy as np

# data paths
data_path = '/groups/scicompsoft/home/fleishmang/rnr-exm-work/validation_data/zebrafish_pair4.h5'
hdf5_file = h5py.File(data_path, 'r')
fix = hdf5_file['/fixed'][...]
mov = hdf5_file['/move'][...]

# segment paths
data_path = '/groups/scicompsoft/home/fleishmang/rnr-exm-work/validation_data/zebrafish_pair4_segmentation.h5'
hdf5_file_seg = h5py.File(data_path, 'r')
fix_seg = hdf5_file_seg['/fixed'][...]
mov_seg = hdf5_file_seg['/move'][...]

# assumed spacings (there are no provided spacings, this is a problem)
fix_spacing = np.array([0.4, 0.1625, 0.1625])
mov_spacing = np.array([0.4, 0.1625, 0.1625])

# # # write some channels
# nrrd.write('./fix.nrrd', fix.transpose(2,1,0), compression_level=2)
# nrrd.write('./mov.nrrd', mov.transpose(2,1,0), compression_level=2)
# nrrd.write('./fix_seg.nrrd', fix_seg.transpose(2,1,0), compression_level=2)
# nrrd.write('./mov_seg.nrrd', mov_seg.transpose(2,1,0), compression_level=2)

## Alignment
---

### affine

In [2]:
# define alignment
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform

# define alignment steps
common_kwargs = {
    'alignment_spacing':4.0,
    'shrink_factors':(2,),
    'smooth_sigmas':(2.,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':400,
    },
}

affine_kwargs = {
    'initial_condition':'CENTER',
}

steps = [('affine', {**common_kwargs, **affine_kwargs},)]

In [3]:
%%time

# run it
affine = alignment_pipeline(
    fix, mov,
    fix_spacing, mov_spacing,
    steps,
)

LEVEL:  0  ITERATION:  0  METRIC:  -0.6786281870326064
LEVEL:  0  ITERATION:  1  METRIC:  -0.6814296949551283
LEVEL:  0  ITERATION:  2  METRIC:  -0.6818059618274652
LEVEL:  0  ITERATION:  3  METRIC:  -0.6802746401267152
LEVEL:  0  ITERATION:  4  METRIC:  -0.6812285727464811
LEVEL:  0  ITERATION:  5  METRIC:  -0.6931605122255029
LEVEL:  0  ITERATION:  6  METRIC:  -0.7083892788654628
LEVEL:  0  ITERATION:  7  METRIC:  -0.7225021801068576
LEVEL:  0  ITERATION:  8  METRIC:  -0.7357865190707356
LEVEL:  0  ITERATION:  9  METRIC:  -0.7479700550739582
LEVEL:  0  ITERATION:  10  METRIC:  -0.7583174147817233
LEVEL:  0  ITERATION:  11  METRIC:  -0.7669753759390571
LEVEL:  0  ITERATION:  12  METRIC:  -0.7776138285951582
LEVEL:  0  ITERATION:  13  METRIC:  -0.7928044759402728
LEVEL:  0  ITERATION:  14  METRIC:  -0.8131627066075591
LEVEL:  0  ITERATION:  15  METRIC:  -0.8374369645537433
LEVEL:  0  ITERATION:  16  METRIC:  -0.8648747973233973
LEVEL:  0  ITERATION:  17  METRIC:  -0.8931600418693971
LE

In [5]:
# apply transform and save results
affine_aligned = apply_transform(
    fix, mov,
    fix_spacing, mov_spacing,
    transform_list=[affine,],
)

# apply transform and save results
affine_aligned_seg = apply_transform(
    fix_seg, mov_seg,
    fix_spacing, mov_spacing,
    transform_list=[affine,],
    interpolate_with_nn=True,
)

# write results
np.savetxt('affine.mat', affine)
nrrd.write('./affine.nrrd', affine_aligned.transpose(2,1,0), compression_level=2)
nrrd.write('./affine_seg.nrrd', affine_aligned_seg.transpose(2,1,0), compression_level=2)

# load precomputed results
affine = np.loadtxt('./affine.mat')

In [2]:
from bigstream.utility import matrix_to_displacement_field

# load affine, convert to displacement, convert to voxel units
affine = np.loadtxt('./affine.mat')
deform = matrix_to_displacement_field(affine, fix.shape, spacing=fix_spacing)
deform = deform / fix_spacing

# save as h5 file
with h5py.File('./zebrafish_val.h5', 'w') as f:
    dset = f.create_dataset('pair4', deform.shape, deform.dtype)
    dset[...] = deform